# Yeast GEM Exploration and Community Modeling

**Author: T. Ruokokoski**

This notebook loads and inspects genome-scale metabolic models (GEMs) of various yeast strains. It prints basic model statistics, simulates flux distributions using Flux Balance Analysis (FBA), and sets up a framework for building a multi-strain community model for bioreactor simulation.

In [13]:
import os
import cobra
from cobra.io import read_sbml_model
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
import pandas as pd
from IPython.display import display

# Set paths
model_dir = "./models"

## Load GEMs

Load genome-scale models (e.g., **yeast9**, <i>(other possibilities: Y. lipolytica, C. intermedia, ...)</i>) from SBML files.


In [14]:
model_paths = {
    "yeast9": os.path.join(model_dir, "yeast-GEM.xml"),
    # Add more as needed
}

models = {name: read_sbml_model(path) for name, path in model_paths.items()}


## Basic Model Summary

Print statistics: number of reactions, metabolites, and genes for each model.

In [15]:
for name, model in models.items():
    print(f"--- {name.upper()} ---")
    print(f"Reactions: {len(model.reactions)}")
    print(f"Metabolites: {len(model.metabolites)}")
    print(f"Genes: {len(model.genes)}")
    display(model.summary())

--- YEAST9 ---
Reactions: 4131
Metabolites: 2806
Genes: 1161


Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.5493,0,0.00%
s_0565,r_1714,1,6,100.00%
s_0796,r_1832,0.02443,0,0.00%
s_0925,r_1861,2.696E-06,0,0.00%
s_1277,r_1992,2.326,0,0.00%
s_1324,r_2005,0.4297,0,0.00%
s_1374,r_2020,0.0003116,0,0.00%
s_1438,r_2049,0.0003408,0,0.00%
s_1468,r_2060,0.007383,0,0.00%
s_4200,r_4593,0.0001107,0,0.00%


## Extract and Display Selected Metabolites and Reactions

This section displays detailed information for selected metabolites and reactions involved in the model.

In [16]:
metabolite_ids = [
    "s_0420", "s_0565", "s_0796", "s_0925", "s_1277",
    "s_1324", "s_1374", "s_1438", "s_1468",
    "s_4200", "s_4201", "s_4202", "s_4203", "s_4204", "s_4199",
    "s_0458", "s_0776", "s_0805", "s_0450", "s_4157"
]
reaction_ids = [
    "r_1654", "r_1714", "r_1832", "r_1861", "r_1992", "r_2005", "r_2020",
    "r_2049", "r_2060", "r_4593", "r_4594", "r_4595", "r_4596", "r_4597",
    "r_4600", "r_1672", "r_1814", "r_2100", "r_2111", "r_4527"
]

# Extract data
metabolite_info = [
    (m.id, m.name, m.formula, m.compartment) 
    for m in model.metabolites if m.id in metabolite_ids
]
reaction_info = [
    (r.id, r.name, r.reaction) 
    for r in model.reactions if r.id in reaction_ids
]

# Convert to dataframes
metabolite_df = pd.DataFrame(metabolite_info, columns=["ID", "Name", "Formula", "Compartment"])
reaction_df = pd.DataFrame(reaction_info, columns=["ID", "Name", "Equation"])

display(metabolite_df.style.set_caption("Metabolites Involved"))
display(reaction_df.style.set_caption("Reactions Involved"))

,ID,Name,Formula,Compartment
0,s_0420,ammonium,H4N,e
1,s_0450,biomass,None,c
2,s_0458,carbon dioxide,CO2,e
3,s_0565,D-glucose,C6H12O6,e
4,s_0776,glycolaldehyde,C2H4O2,e
5,s_0796,H+,H,e
6,s_0805,H2O,H2O,e
7,s_0925,iron(2+),Fe,e
8,s_1277,oxygen,O2,e
9,s_1324,phosphate,HO4P,e


,ID,Name,Equation
0,r_1654,ammonium exchange,s_0420 <=>
1,r_1672,carbon dioxide exchange,s_0458 -->
2,r_1714,D-glucose exchange,s_0565 <=>
3,r_1814,glycolaldehyde exchange,s_0776 -->
4,r_1832,H+ exchange,s_0796 <=>
5,r_1861,iron(2+) exchange,s_0925 <=>
6,r_1992,oxygen exchange,s_1277 <=>
7,r_2005,phosphate exchange,s_1324 <=>
8,r_2020,potassium exchange,s_1374 <=>
9,r_2049,sodium exchange,s_1438 <=>


## Inspect Specific Reaction and Metabolite

Helper function for detailed inspection of a specific reaction and metabolite within the model.

In [17]:
def inspect_reaction_and_metabolite(model, reaction_id, metabolite_id):
    # Inspect the reaction
    reaction = model.reactions.get_by_id(reaction_id)
    print("=== Reaction Details ===")
    print(f"ID        : {reaction.id}")
    print(f"Name      : {reaction.name}")
    print(f"Equation  : {reaction.reaction}")
    print(f"GPR       : {reaction.gene_reaction_rule}")

    # Inspect the metabolite
    metabolite = model.metabolites.get_by_id(metabolite_id)
    print("\n=== Metabolite Details ===")
    print(f"ID         : {metabolite.id}")
    print(f"Name       : {metabolite.name}")
    print(f"Formula    : {metabolite.formula}")
    print(f"Compartment: {metabolite.compartment}")
    print("Annotations:")
    for key, value in metabolite.annotation.items():
        print(f"  {key}: {value}")

    # List reactions involving this metabolite
    print("\nReactions involving this metabolite:")
    for r in metabolite.reactions:
        print(f"{r.id}: {r.reaction}")

In [18]:
inspect_reaction_and_metabolite(models["yeast9"], "r_1714", "s_0565")

=== Reaction Details ===
ID        : r_1714
Name      : D-glucose exchange
Equation  : s_0565 <=> 
GPR       : 

=== Metabolite Details ===
ID         : s_0565
Name       : D-glucose
Formula    : C6H12O6
Compartment: e
Annotations:
  sbo: SBO:0000247
  bigg.metabolite: glc__D
  chebi: CHEBI:4167
  kegg.compound: C00031
  metanetx.chemical: MNXM41

Reactions involving this metabolite:
r_0370: s_0003 + s_0805 --> s_0565
r_4400: s_0805 + s_4140 --> s_0565 + s_1106
r_1714: s_0565 <=> 
r_1166: s_0565 --> s_0563
r_4420: s_0805 + s_4131 <=> s_0554 + s_0565
r_1024: s_0805 + s_1466 --> s_0554 + s_0565


In [19]:
# Inspect biomass growth
inspect_reaction_and_metabolite(models["yeast9"], "r_2111", "s_0450")

=== Reaction Details ===
ID        : r_2111
Name      : growth
Equation  : s_0450 --> 
GPR       : 

=== Metabolite Details ===
ID         : s_0450
Name       : biomass
Formula    : None
Compartment: c
Annotations:
  sbo: SBO:0000649
  bigg.metabolite: biomass

Reactions involving this metabolite:
r_2111: s_0450 --> 
r_4041: 55.3 s_0434 + 55.3 s_0803 + s_1096 + s_3717 + s_3718 + s_3719 + s_3720 + s_4205 + s_4206 --> 55.3 s_0394 + s_0450 + 55.3 s_0794 + 55.3 s_1322


## Flux Balance Analysis (FBA)

Simulate optimal flux distribution for each yeast model using default objective.

In [20]:
# FBA for yeast9
model = models["yeast9"]
print("FBA for yeast9")
print("-" * 40)
solution = model.optimize()
print(f"Objective function   : {model.objective.expression}")
print(f"Objective direction  : {model.objective.direction}")
print(f"Objective value      : {solution.objective_value:.4f}")

print("\nTop flux-carrying reactions:")
print(solution.fluxes.nlargest(5))

print(f"\nNumber of exchange reactions: {len(model.exchanges)}")
print("Example exchange reactions:")
for rxn in model.exchanges[:5]:
    print(f"{rxn.id}: {rxn.reaction}")

FBA for yeast9
----------------------------------------
Objective function   : 1.0*r_2111 - 1.0*r_2111_reverse_58b69
Objective direction  : max
Objective value      : 0.0858

Top flux-carrying reactions:
r_0438    9.116049
r_1110    6.369549
r_0226    5.731505
r_1245    5.124964
r_0439    4.558025
Name: fluxes, dtype: float64

Number of exchange reactions: 271
Example exchange reactions:
r_1542: s_0003 --> 
r_1545: s_0022 --> 
r_1546: s_0026 --> 
r_1547: s_0029 --> 
r_1548: s_0032 --> 


## Parsimonius Flux Balance Analysis (pFBA)

pFBA finds a flux distribution which gives the optimal growth rate, but minimizes the total sum of flux. Both pFBA and FBA should return identical results within solver tolerances for the objective being optimized.

In [21]:
original_objective = model.objective

# Switch to single reaction objective
model.objective = "r_2111"
fba_solution = model.optimize()
pfba_solution = pfba(model)

diff = abs(fba_solution.fluxes["r_2111"] - pfba_solution.fluxes["r_2111"])
print(f"Difference in flux through reaction 'r_2111' between standard FBA and pFBA: {diff}")

# Restore original objective
model.objective = original_objective

Difference in flux through reaction 'r_2111' between standard FBA and pFBA: 0.0


## Identify NADH Metabolites and Inspect Flux

List all metabolites whose name includes "NADH" and then inspect the fluxes involving cytosolic NADH (s_1203) by printing summary. This can be used to examine the overall redox balance of the model.

**Note:** NADH (Nicotinamide Adenine Dinucleotide, reduced form) is a key electron carrier in cellular metabolism.

In [22]:
# Find NADH metabolite IDs
for m in model.metabolites:
    if "nadh" in m.name.lower():
        print(m.id, m.name, m.compartment)

model.metabolites.s_1203.summary()

s_1203 NADH c
s_1204 NADH er
s_1205 NADH m
s_1206 NADH p
s_2818 NADH erm
s_3753 NADH n


Percent,Flux,Reaction,Definition
1.88%,0.02923,r_0061,s_0009 + s_1198 --> s_0010 + s_0794 + s_1203
0.00%,1.631E-05,r_0172,s_0208 + s_0803 + s_1198 --> s_0441 + 2.0 s_0794 + s_1203
0.20%,0.003166,r_0235,s_0297 + s_1198 --> s_0209 + s_0456 + s_1203
0.26%,0.003966,r_0445,s_0722 + s_1198 --> s_0456 + s_1203
89.28%,1.386,r_0486,s_0764 + s_1198 + s_1322 <=> s_0075 + s_0794 + s_1203
0.84%,0.01308,r_0536,s_0803 + s_1010 + 2.0 s_1198 --> 3.0 s_0794 + s_1006 + 2.0 s_1203
0.27%,0.00412,r_0565,s_0803 + s_0849 + s_1198 --> s_0794 + s_1203 + s_1565
4.27%,0.06624,r_0891,s_0260 + s_1198 --> s_0258 + s_0794 + s_1203
1.82%,0.02822,r_0988,s_0803 + s_1038 + s_1198 --> s_0180 + s_0794 + s_1025 + s_1203
1.18%,0.01833,r_4581,s_1198 + s_4189 --> s_0178 + s_0456 + s_1203


## Identify ATP Metabolites and Inspect Flux

Lists all metabolites with "ATP" in their name along with their compartments, helping identify the relevant ATP species in the model. We then focus on the cytosolic ATP (s_0434) to get an overview of the main reactions producing and consuming ATP.

In [23]:
# Find ATP metabolite IDs
for m in model.metabolites:
    if "atp" in m.name.lower():
        print(m.id, m.name, m.compartment)

model.metabolites.s_0434.summary()

s_0326 5-phosphoribosyl-ATP c
s_0434 ATP c
s_0435 ATP er
s_0437 ATP m
s_0438 ATP n
s_0439 ATP p
s_0586 dATP c
s_2831 ATP erm
s_2840 ATP lp
s_2856 ATP ce
s_3341 ATP vm
s_3359 ATP gm
s_3881 ATP v
s_4196 ATP g
s_4318 dATP m


Percent,Flux,Reaction,Definition
0.00%,0.000206,r_0330,s_0434 + s_0615 <=> s_0394 + s_0613
15.16%,1.386,r_0892,s_0075 + s_0394 <=> s_0260 + s_0434
13.86%,1.267,r_0962,s_0394 + s_0794 + s_1360 --> s_0434 + s_1399
70.96%,6.487,r_1110,s_0394 + s_0437 <=> s_0397 + s_0434
0.01%,0.0005151,r_1704,s_0434 + s_0589 <=> s_0394 + s_0587
0.00%,0.000309,r_1729,s_0434 + s_0584 <=> s_0394 + s_0582
0.01%,0.0005014,r_3543,s_0434 <=> s_2831
Percent,Flux,Reaction,Definition
0.10%,-0.008791,r_0079,s_0301 + s_0434 + s_0803 + s_0999 --> s_0302 + s_0394 + s_0794 + s_0991 + s_1322
1.11%,-0.1016,r_0109,s_0373 + s_0434 + s_0445 --> s_0394 + s_0794 + s_1101 + s_1322


## Set Up Community Model

Prepare compartments, exchange reactions, and initial setup for community simulation.

In [24]:
community_model = Model("microbial_community")

# TODO